In [3]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="white")
import os
import plotly.express as px
from plotly.subplots import make_subplots
from plotly import graph_objects as go
import warnings
warnings.filterwarnings("ignore")

Note: in this case when we refer to "true" with respect to the regime, what we mean is that we allow the controller to control the f1tenth in all experiments. Proof of this can be found in the following launch file [sim_for_rtreach_aij_experiments.launch](https://github.com/pmusau17/Platooning-F1Tenth/blob/noetic-port/src/race/launch/sim_for_rtreach_aij_experiments.launch) in lines 104-107 

```
<!--Reachability safety node that incorporates dynamic obstacles-->
    <node pkg="rtreach" unless="$(arg control)" type="reach_node_uncertain_simplex" output="screen" name="reach_node_simplex"  args="$(arg wall_time) $(arg reach_time) $(arg box_display_limit) $(arg parameter_uncertainty) $(arg algorithm_name) $(arg number_of_cars) $(arg num_obstacles) $(arg position_uncertainty) $(arg velocity_uncertainty)" required="true"/>

    <node pkg="rtreach" if="$(arg control)" type="reach_node_no_simplex" output="screen" name="reach_node_simplex"  args="$(arg wall_time) $(arg reach_time) $(arg box_display_limit) $(arg parameter_uncertainty) $(arg algorithm_name) $(arg number_of_cars) $(arg num_obstacles) $(arg position_uncertainty) $(arg velocity_uncertainty)" required="true"/>
```

if the argument **control** is true, the simplex node will not be run, and the reach_node_no_simplex will be run

In [74]:
original_dfs = []
opponents = [2,3]
obstacles = [0,5]

control = ["true","false"]
velocities = ["0.5","1.0","1.5"]
velocities_fn = {"0.5":"0.500000", "1.0":"1.000000","1.5":"1.500000"}
controllers= ["disparity_extender","pure_pursuit","e2e"]
file_str = "{}_{}_1.000000_{}_{}_0.0_0.0_{}.csv"
collision_str = "collisions_{}_{}_{}_{}_{}_cones.csv"
for cntrl in controllers:
    for opp in opponents:
        for obs in obstacles:
            for vel in velocities:
                for regime in control:
                    
                    
                    if(regime=="true"):
                        file_name = "nosimplex"
                    else:
                        file_name = "simplex"
                        
                    
                    fh = file_str.format(file_name,cntrl,opp,obs,velocities_fn[vel])
                    ch = collision_str.format(cntrl,opp,obs,vel,regime)
                    
                    try:
                        df = pd.read_csv(fh,header=None).tail(5) # I only want the most recently run experiments here
                        df =df.rename(columns={0:'time_str',1:'time_taken_lec',2:'time_taken_safety_controller',3:'area'})
                        df['controller'] = cntrl
                        df['time_taken_lec'] * 100
                        df['number_of_opponents'] = opp
                        df['obstacle_presence']= obs>0
                        df['regime'] = regime
                        df['velocity'] = vel
                        
                        df = df.drop(columns = ["time_taken_safety_controller","area"])
                        
                        if(os.path.exists(ch)):
                            c_df = pd.read_csv(ch,header=None)
                            df['collision_percentage'] = ((c_df.count()[0])/5)*100
                        else:
                            df['collision_percentage'] = 0.0
                        
                        original_dfs.append(df)
                    except:
                        continue

                        
dfs = pd.concat(original_dfs)
dfs
                    
                    


,time_str,time_taken_lec,controller,number_of_opponents,obstacle_presence,regime,velocity,collision_percentage
5,01/04/2022/18:23:04,0.876521,disparity_extender,2,False,true,0.5,0.0
6,01/04/2022/18:25:14,0.864547,disparity_extender,2,False,true,0.5,0.0
7,01/04/2022/18:27:24,0.876153,disparity_extender,2,False,true,0.5,0.0
8,01/04/2022/18:29:35,0.875509,disparity_extender,2,False,true,0.5,0.0
9,01/04/2022/18:31:43,0.889403,disparity_extender,2,False,true,0.5,0.0
...,...,...,...,...,...,...,...,...
5,02/04/2022/03:07:42,0.191453,e2e,3,True,false,1.5,0.0
6,02/04/2022/03:10:51,0.187395,e2e,3,True,false,1.5,0.0
7,02/04/2022/03:14:01,0.148431,e2e,3,True,false,1.5,0.0
8,02/04/2022/03:17:13,0.235294,e2e,3,True,false,1.5,0.0


In [75]:
dfs[dfs['collision_percentage'] > 0]

,time_str,time_taken_lec,controller,number_of_opponents,obstacle_presence,regime,velocity,collision_percentage
5,01/04/2022/18:33:35,0.178045,disparity_extender,2,False,false,0.5,100.0
6,01/04/2022/18:35:26,0.173962,disparity_extender,2,False,false,0.5,100.0
7,01/04/2022/18:37:19,0.179240,disparity_extender,2,False,false,0.5,100.0
8,01/04/2022/18:39:09,0.175054,disparity_extender,2,False,false,0.5,100.0
9,01/04/2022/18:41:00,0.174151,disparity_extender,2,False,false,0.5,100.0
...,...,...,...,...,...,...,...,...
5,02/04/2022/03:01:25,0.171429,e2e,3,True,true,1.5,100.0
6,02/04/2022/03:02:10,0.100000,e2e,3,True,true,1.5,100.0
7,02/04/2022/03:02:55,0.144737,e2e,3,True,true,1.5,100.0
8,02/04/2022/03:03:39,0.062500,e2e,3,True,true,1.5,100.0


### Safety Analysis

Summary of all of experiments without the use of the simplex architechture, this gives us a rough sense of the performance of each controller.

In [76]:
sa = dfs[dfs["regime"]=="true"]
#sa = dfs.drop(columns=["regime"])
sa.groupby(["controller","number_of_opponents","obstacle_presence","velocity"]).agg(["mean","std"])

time_taken_lec  \
                                                                            mean   
controller         number_of_opponents obstacle_presence velocity                  
disparity_extender 2                   False             0.5            0.876427   
                                                         1.0            0.692064   
                                                         1.5            0.717144   
                                       True              0.5            0.875328   
                                                         1.0            0.710232   
                                                         1.5            0.616113   
                   3                   False             0.5            0.921718   
                                                         1.0            0.658406   
                                                         1.5            0.668865   
                                       True              0.5            0.909887   
                                                         1.0            0.635607   
                                                         1.5            0.651068   
e2e                2                   False             0.5            0.951636   
                                                         1.0            0.168498   
                                                         1.5            0.245323   
                                       True              0.5            0.953581   
                                                         1.0            0.751367   
                                                         1.5            0.624695   
                   3                   False             0.5            0.997144   
                                                         1.0            0.359900   
                                                         1.5            0.110916   
                                       True              0.5            0.974949   
                                                         1.0            0.371350   
                                                         1.5            0.159622   
pure_pursuit       2                   False             0.5            1.000000   
                                                         1.0            0.904349   
                                                         1.5            0.948548   
                                       True              0.5            0.965559   
                                                         1.0            0.723505   
                                                         1.5            0.828641   
                   3                   False             0.5            1.000000   
                                                         1.0            0.121434   
                                                         1.5            0.732695   
                                       True              0.5            0.967915   
                                                         1.0            0.078915   
                                                         1.5            0.338553   

                                                                             \
                                                                        std   
controller         number_of_opponents obstacle_presence velocity             
disparity_extender 2                   False             0.5       0.008810   
                                                         1.0       0.133244   
                                                         1.5       0.012697   
                                       True              0.5       0.004999   
                                                         1.0       0.012094   
                                                         1.5       0.112636   
                   3                   False             0.5       0.006190   

### Simplex

In [78]:
simplex = dfs[dfs["regime"]=="false"]
simplex.groupby(["controller","number_of_opponents","obstacle_presence","velocity"]).agg(["mean","std"])

time_taken_lec  \
                                                                            mean   
controller         number_of_opponents obstacle_presence velocity                  
disparity_extender 2                   False             0.5            0.176090   
                                                         1.0            0.247274   
                                                         1.5            0.158714   
                                       True              0.5            0.174709   
                                                         1.0            0.251083   
                                                         1.5            0.157594   
                   3                   False             0.5            0.715132   
                                                         1.0            0.211217   
                                                         1.5            0.133934   
                                       True              0.5            0.746348   
                                                         1.0            0.206900   
                                                         1.5            0.169297   
e2e                2                   False             0.5            0.917066   
                                                         1.0            0.433616   
                                                         1.5            0.298754   
                                       True              0.5            0.802429   
                                                         1.0            0.477446   
                                                         1.5            0.278882   
                   3                   False             0.5            0.977840   
                                                         1.0            0.406871   
                                                         1.5            0.382537   
                                       True              0.5            0.930162   
                                                         1.0            0.336649   
                                                         1.5            0.188259   
pure_pursuit       2                   False             0.5            1.000000   
                                                         1.0            0.883302   
                                                         1.5            0.599391   
                                       True              0.5            0.939474   
                                                         1.0            0.743729   
                                                         1.5            0.625160   
                   3                   False             0.5            1.000000   
                                                         1.0            0.672382   
                                                         1.5            0.507657   
                                       True              0.5            0.941669   
                                                         1.0            0.605043   
                                                         1.5            0.210414   

                                                                             \
                                                                        std   
controller         number_of_opponents obstacle_presence velocity             
disparity_extender 2                   False             0.5       0.002403   
                                                         1.0       0.009674   
                                                         1.5       0.005560   
                                       True              0.5       0.001057   
                                                         1.0       0.007225   
                                                         1.5       0.004019   
                   3                   False             0.5       0.053849   

### No Simplex

In [70]:
no_simplex = dfs[dfs["regime"]=="false"]
no_simplex.groupby(["controller","number_of_opponents","obstacle_presence","velocity"]).agg(["mean","std"])

time_taken_lec  \
                                                                            mean   
controller         number_of_opponents obstacle_presence velocity                  
disparity_extender 2                   False             0.5            0.876427   
                                                         1.0            0.692064   
                                                         1.5            0.717144   
                                       True              0.5            0.875328   
                                                         1.0            0.710232   
                                                         1.5            0.616113   
                   3                   False             0.5            0.921718   
                                                         1.0            0.658406   
                                                         1.5            0.668865   
                                       True              0.5            0.909887   
                                                         1.0            0.635607   
                                                         1.5            0.651068   
e2e                2                   False             0.5            0.951636   
                                                         1.0            0.168498   
                                                         1.5            0.245323   
                                       True              0.5            0.953581   
                                                         1.0            0.751367   
                                                         1.5            0.624695   
                   3                   False             0.5            0.997144   
                                                         1.0            0.359900   
                                                         1.5            0.110916   
                                       True              0.5            0.974949   
                                                         1.0            0.371350   
                                                         1.5            0.159622   
pure_pursuit       2                   False             0.5            1.000000   
                                                         1.0            0.904349   
                                                         1.5            0.948548   
                                       True              0.5            0.965559   
                                                         1.0            0.723505   
                                                         1.5            0.828641   
                   3                   False             0.5            1.000000   
                                                         1.0            0.121434   
                                                         1.5            0.732695   
                                       True              0.5            0.967915   
                                                         1.0            0.078915   
                                                         1.5            0.338553   

                                                                             \
                                                                        std   
controller         number_of_opponents obstacle_presence velocity             
disparity_extender 2                   False             0.5       0.008810   
                                                         1.0       0.133244   
                                                         1.5       0.012697   
                                       True              0.5       0.004999   
                                                         1.0       0.012094   
                                                         1.5       0.112636   
                   3                   False             0.5       0.006190   